In [ ]:
#QUANTT TSLA Option Pricing - Monte Carlo

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import tqdm

# Initial stock conditions
Stock_P = 9.9867  # Initial stock price
Strike_P = 10.48083960978591  # Strike price
vol = 0.5527  # Implied volatility
r = 0.03  # Risk-free rate
N = 252  # Number of time steps
M = 100000  # Number of simulations
T = 21 / 365  # Time to expiration in years

# Constants for the formula
dt = T / N
u_dt = (r - 0.5 * vol ** 2) * dt
vol_sqrt_dt = vol * np.sqrt(dt)

# Function to sample from the KDE
def sample_from_kde(kde, n=1):
    '''Sample from the KDE.'''
    return kde.resample(n)[0][0]

# Monte Carlo Simulation using KDE for price changes
def monte_carlo_simulation_option_valuation(Stock_P, vol, r, T, N, M, Strike_P): #These are the variables from the monte carlo equation
    # Constants. These apply to the formula -----> delta(x) = u*delta(t) + sigma*delta(z)
    dt = T/N                                #   timestep. I.E., if T is 5 years and N is 10, this will give each time step as 0.5 years
    u_dt = (r-0.5*vol**2)*dt                #   drift term (u)  
    vol_sqrt_dt = vol*np.sqrt(dt)           #   From equation on line 43, delta(z) becomes sqrt(delta(t)) when time step is applied
                                            #   -----> StockPriceNow = StockPricePrevious*e^(u*delta(t) + sigma*delta(z))

    simulation_paths = [] #store paths for the monte-carlo simulation graph
    simulation_results = [] #store results for the numerical value prediction
    for i in range(M):  # M is total simulations
        path = [Stock_P] #used for mapping the paths
        lnStock_P = np.log(Stock_P) #used for numerical option value
        for _ in range(N):  # N is number of time steps
            # Sample a change using KDE method (assuming KDE is fitted outside)
            z = np.random.normal()  # Placeholder for KDE sample
            lnStock_P += (u_dt + vol_sqrt_dt * z)
            path.append(np.exp(lnStock_P)) #stores the path
        final_stock_price = np.exp(lnStock_P)
        # Calculating the option value at expiration
        option_val = max(final_stock_price - Strike_P, 0)
        simulation_results.append(option_val) #add each result (numerical) to results list
        simulation_paths.append(path) #add each path to path list
    
    # Average the simulated option values
    average_option_val = np.mean(simulation_results)
    return simulation_results, average_option_val, simulation_paths

# Assuming KDE is defined and fitted outside this block. Replace z with sample_from_kde(kde) in actual implementation.

# Example call to the function (without a fitted KDE, thus using a normal distribution as placeholder)
simulation_results, average_option_val, simulation_paths = monte_carlo_simulation_option_valuation(Stock_P, vol, r, T, N, M, Strike_P)
print(f"Average Option Value: {average_option_val}")

#Plot ditribution graph
plt.figure(figsize=(12, 8))
plt.hist(simulation_results, bins=50, alpha=0.75)
plt.title('Distribution of Simulated Option Values')
plt.xlabel('Option Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

#Plot monte carlo simulation graph
plt.figure(figsize=(12, 8))
for simulation in simulation_paths[:1000]:  #Plot only the first 1000 simulations for clarity
    plt.plot(simulation, lw=1, alpha=0.2)

plt.title('Monte Carlo Simulation of TSLA Option Prices')
plt.xlabel('Time Steps (1 Step = 1 Trading Day)')
plt.ylabel('Simulated Option Price (USD)')
plt.grid(True)
plt.show()

print(simulation_paths)